# `metadsl`

...is a framework for creating domain specific languages in Python.

In [1]:
from __future__ import annotations

from metadsl import *
from metadsl_core import *
from metadsl_visualize import *


Here, we show a NumPy API compatible DSL that we have started writing. We can create a symbolic expression and then execute it. The widget shows the different stages of execution:

In [2]:
N = 100
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'-4274288441019960025': ['8002953247574304563', PrimitiveNode(type='int', repr='100')], '1858995438532015666': ['1858995438532015666', CallNode(function='arange', type_params=None, args=['-4274288441019960025'], kwargs=None)], '-6642729776650570191': ['-6642729776650570191', CallNode(function='NDArrayCompat.__add__', type_params=None, args=['1858995438532015666', '1858995438532015666'], kwargs=None)], '-4274288440914955100': ['-5287040927963181481', PrimitiveNode(type='int', repr='5')], '-8761188906761391537': ['-8761188906761391537', CallNode(function='NDArrayCompat.__getitem__', type_params=None, args=['-6642729776650570191', '-4274288440914955100'], kwargs=None)], '7479098326220010278': ['7479098326220010278', CallNode(function='NDArrayCompat.to_ndarray', type_params=None, args=['-8761188906761391537'], kwargs=None)], '-4154059725933008488': ['-4154059725933008488', CallNode(function='Converter.convert', type_params={'T': DeclaredTypeInstance(type='NDA

array(10)

Now, we can add an optimization replacement, so that we do the getitem before adding the two expressions:

In [3]:
@register
@rule
def optimize_getitem_add(l: NDArray, r: NDArray, idx: IndxType) -> R[NDArray]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

Now, when we look at the execution steps, we see that it is doing each getitem before adding, which should be more efficient:

In [4]:
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

expr._ipython_display_()

Typez(definitions=None, nodes={'-8709756823428538841': ['4924960356701885191', PrimitiveNode(type='int', repr='100')], '-3416292616964293722': ['-3416292616964293722', CallNode(function="arange: <class 'metadsl_core.numpy.NDArrayCompat'>", type_params=None, args=['-8709756823428538841'], kwargs=None)], '5008603041503392762': ['5008603041503392762', CallNode(function="NDArrayCompat.__add__: <class 'metadsl_core.numpy.NDArrayCompat'>", type_params=None, args=['-3416292616964293722', '-3416292616964293722'], kwargs=None)], '-8709756823323533916': ['-6412702131201121431', PrimitiveNode(type='int', repr='5')], '-3861742383636140600': ['-3861742383636140600', CallNode(function="NDArrayCompat.__getitem__: <class 'metadsl_core.numpy.NDArrayCompat'>", type_params=None, args=['5008603041503392762', '-8709756823323533916'], kwargs=None)], '-3306277675734339598': ['-3306277675734339598', CallNode(function='NDArrayCompat.to_ndarray: metadsl.expressions.PlaceholderExpression[numpy.ndarray]', type_pa

10

This shows how we can break up the NumPy API into different layers, all of which are extensible:

1. A compatibility layer that works like the existing NumPy API, except isn't limited to the Python types of the current API
2. A type safe version of this API. The conversion between the compatability layer and this layer is extensible, so that third party authors can add new conversion between their own Python objects and the typed representation.
3. A backend layer that translates either back to Python calls or source code, or to other targets like LLVM or Tensorflow.

The key is that all these layers are composable, so you could have different frontends for any of them or add your own. This is all done through a typed replacement system that is compatible with static analysis using MyPy.

`metadsl` is the glue that allows us to define this API in a way that enables greater greater collaboration. Since each layer has it's own well defined specification, different projects can extend and target these seperate layers, while minimizing the need to explicitly opt in to collaborate. The goal is to enable greater reuse, interoperability, and extensability within the Python scientific ecosystem.